# Reproduce speedup figures

## Imports

In [ ]:
import numpy as np
from generator import *
from solver import *
from visualize import *
from speedup_analysis import *
import warnings
warnings.filterwarnings("ignore")

## Load model for analysis

In [ ]:
import  tensorflow
import os
from ipywidgets import *
import time

thisdir = 'SavedModels'
models = []
loaded_model =''
for r, d, f in os.walk(thisdir):
    for file in f:
        if file.endswith(".h5"):
            models.append(os.path.join(r, file))

@interact(models=models)
def load_model(models):
    global loaded_model 
    loaded_model = tensorflow.keras.models.load_model(models,custom_objects={'LeakyReLU':tensorflow.keras.layers.LeakyReLU()})
    print(f'{models} loaded !!')

## Plot format

In [ ]:
import matplotlib.pyplot as plt
import tensorflow
import numpy as np
import matplotlib as mpl
mpl.rcParams.update({
    "text.usetex": False,  # True if latex is installed
    "font.family": "serif",
    "font.serif": ["dejavuserif"]})

mpl.rcParams['figure.dpi'] = 72
mpl.rcParams.update({'font.size': 35})
mpl.rcParams['axes.linewidth'] = 2
from matplotlib import ticker
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)


'''
If latex is installed on the system -> uncomment

'''
# import matplotlib.pyplot as plt
# # import torch
# import tensorflow
# import numpy as np
# import matplotlib as mpl
# mpl.rcParams.update({
#     "text.usetex": False,  # True if latex is installed
#     "font.family": "sans-serif",
#     "font.sans-serif": ["Helvetica"]})

# mpl.rcParams['figure.dpi'] = 72
# mpl.rcParams.update({'font.size': 35})
# mpl.rcParams['axes.linewidth'] = 2
# from matplotlib import ticker
# formatter = ticker.ScalarFormatter(useMathText=True)
# formatter.set_scientific(True)

### Speedup plots

In [ ]:
@interact(speedup_analysis=['Iteration analysis','Grid Size analysis'],P=[10,100],grid_size=[12,24,48,96,192,960],start=False)
def speedup_analysis(speedup_analysis,P,grid_size,start):
    
    if speedup_analysis == 'Iteration analysis' and start:
        '''
        Fixed grid size for iteration analysis = 960
        P = prediction per step
        K = number of deeplearning prediction
        '''
        
        print('Analysis...')
        iteration_solution,iteration_prediction,iteration_logs = iterations_speedup_analysis(loaded_model,grid_size=grid_size,P=P,K=10,max_iter=20)
        
        print('Plotting...')
        plt.figure(figsize=(15,5))

        iterations = np.array([ key for key in iteration_logs ] )
        MAE_iterations = np.array([ iteration_logs[key]['MAE'] for key in iteration_logs ])
        speedup_iterations = np.array([ iteration_logs[key]['NumericalTime']/iteration_logs[key]['PredictionTime'] for key in iteration_logs ])


        m,b = np.polyfit(iterations,speedup_iterations,1)

        fig, ax1 = plt.subplots(figsize=(15,7))

        ax1.title.set_text(f'$Iterations - Speedup \ for \ P = 100$')
        ax1.plot(iterations,m*iterations+b,'-k',linewidth=3)
        ax1.scatter(iterations,speedup_iterations,color='black')
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.set_ylabel('$Speedup$', color='black')
        ax1.set_xlabel('$Iterations $', color='black')

        ax2 = ax1.twinx()  
        ax2.set_ylabel('$MAE$', color='red')  
        ax2.plot(iterations, MAE_iterations,'-r',linewidth=3)
        ax2.tick_params(axis='y', labelcolor='red')

        fig.tight_layout()
        
    elif speedup_analysis == 'Grid Size analysis' and start :
        print('Analysis...')
        grid_solution,grid_prediction,grid_logs = gridsize_speedup_analysis(loaded_model,P=P,K=10,grids=np.arange(96,500,96))
        
        plt.figure(figsize=(15,5))
        grids = np.arange(96,500,96)
        MAE_grids= np.array([ grid_logs[key]['MAE'] for key in grid_logs ])
        speedup_grids=np.array([grid_logs[key]['NumericalTime'] / grid_logs[key]['PredictionTime'] for key in grid_logs] )


        fig, ax1 = plt.subplots(figsize=(15,7))

        ax1.title.set_text(f'$Grid \ size - Speedup \ for \ P = 100$')
        ax1.plot(grids,speedup_grids,'-k',linewidth=3)
        ax1.tick_params(axis='y', labelcolor='black')
        ax1.set_ylabel('$Speedup$', color='black')
        ax1.set_xlabel('$Grid \ size \ NXN$', color='black')

        ax2 = ax1.twinx()  
        ax2.set_ylabel('$MAE$', color='red')  
        ax2.plot(grids, MAE_grids,'-r',linewidth=3)
        ax2.tick_params(axis='y', labelcolor='red')

        fig.tight_layout()  # otherwise the right y-label is slightly clipped
        